# Libraries Import

In [31]:
import pandas as pd

# for connection with Azure SQL Database
import pyodbc
import urllib
import sqlalchemy

# for keeping credentials out of sight
import os
from dotenv import load_dotenv

# Credentials and Authorization

You will need to insert your database connection parameters and save as `sql-keys.env` local file.

```
DB_SERVER = "XXXXX"
DB_NAME = "XXXXX" 
DB_USERNAME = "XXXXX"
DB_PASSWORD = XXXXX
```
The following is the list of the connection parameters:
- *DB_SERVER*: database server address e.g., localhost or an IP address.
- *DB_NAME*: the name of the database that you want to connect.
- *DB_USERNAME*: the username used to authenticate.
- *DB_PASSWORD*: password used to authenticate.


In [32]:
# establish working directory path
# getcwd() returns current working directory
wdir_path = os.getcwd()

sql_path = os.path.join(wdir_path, "sql-keys.env") # absolute path of "sql-keys.env"
# load the credentials into os environment 
load_dotenv(sql_path)
# check if credentials loaded successfully
os.environ

# getting credentials information from "sql-keys.env"
server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

driver = ''
# retriving ODBC Driver information from user's computer using Pyodbc
driver_names = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
if driver_names:
    driver = driver_names[0]

# Connection to Database

In [45]:
try:
    # if driver exists, create connection with Azure SQL Database
    if driver:
        conn_string = "DRIVER={" + driver + "};SERVER=" + server + ";DATABASE=" + database + ";UID=" + username + ";PWD=" + password
                
        # According to SQLAlchemy's documentation, an exact PyODBC connection string can be
        # sent in pyodbc's format directly using the parameter odbc_connect.
        # As the delimiters need to be URL-encoded (especially the Driver), urllib.parse.quote_plus is used
        # to encode the PyODBC connection string.
        db_params = urllib.parse.quote_plus(conn_string)
        engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(db_params), fast_executemany=True)
        
        # establish the connection
        cnxn = engine.raw_connection()

        # create a cursor object
        cursor = cnxn.cursor()

        print("Connection to database created successfully.")
    else:
        print("No suitable driver found. Cannot connect.")
except Exception as e:
    print("Connection could not be made due to the following error: \n", e)

Connection to database created successfully.


# Create Tables into Database

In [37]:
def create_table(command, msg):
    '''
    Function to create table into Azure database
    
    Parameters:
        command (string): SQL statement to create new table in database
        msg (string): A string message to print that table is successfully created
    '''
    try:
        cursor.execute(command)
        cnxn.commit()     
    except Exception as e:
        # rollback current transaction if there is an error
        cnxn.rollback()
        raise e
    print(msg)

# create olist_sellers table
create_table("""
            IF OBJECT_ID('dbo.olist_sellers', 'U') IS NULL 
            BEGIN
                CREATE TABLE olist_sellers
                (
                    seller_id VARCHAR(255) PRIMARY KEY NOT NULL, 
                    seller_zip_code_prefix INT, 
                    seller_city VARCHAR(255), 
                    seller_state VARCHAR(2)
                )
            END;
            """,
            "olist_sellers successfully created.")

# create olist_customers table
create_table("""
            IF OBJECT_ID('dbo.olist_customers', 'U') IS NULL 
            BEGIN
                CREATE TABLE olist_customers 
                (
                    customer_id VARCHAR(255) PRIMARY KEY NOT NULL,
                    customer_unique_id VARCHAR(255) NOT NULL, 
                    customer_zip_code_prefix INT, 
                    customer_city VARCHAR(255), 
                    customer_state VARCHAR(2)
                )
            END;
            """,
            "olist_customers successfully created.")

# create olist_geolocation
create_table("""
            IF OBJECT_ID('dbo.olist_geolocation', 'U') IS NULL 
            BEGIN
                CREATE TABLE olist_geolocation 
                (
                    geolocation_zip_code_prefix INT PRIMARY KEY NOT NULL,
                    geolocation_lat DECIMAL NOT NULL, 
                    geolocation_lng DECIMAL NOT NULL
                )
            END;
            """,
            "olist_geolocation successfully created.")

olist_sellers successfully created.
olist_customers successfully created.
olist_geolocation successfully created.


# Data Cleaning

### Customer data

The `olist_customers_dataset` contains location informations of the customer.
- `customer_unique_id` is a unique id identifying customers in the system. It is an id generated at the time of **signup**.
- `customer_id` is a temporary id generated everytime the customer places an order.

In [38]:
# import dataset
cust_df = pd.read_csv("data/olist_customers_dataset.csv")

# view dataset
cust_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [39]:
print("Total number of rows in olist_customers_dataset: " + str(len(cust_df)))

# determine the number of unique values in each column
print("\nNumber of unique values in each column:")
print(cust_df.nunique(axis=0))

Total number of rows in olist_customers_dataset: 99441

Number of unique values in each column:
customer_id                 99441
customer_unique_id          96096
customer_zip_code_prefix    14994
customer_city                4119
customer_state                 27
dtype: int64


# Insert into Azure SQL Database

In [29]:
def write_df(df_name, table_name, msg):
    '''
    Function to insert values from dataframe into Azure database
    
    Parameters:
        df_name (DataFrame): DataFrame name to get data from
        table_name (string): Table name to update in SQL Azure Database
        msg (string): A string message to print that table is successfully updated
    '''
    try:
        df_name.to_sql(table_name, engine, index=False, if_exists="append", schema="dbo")
        cnxn.commit()     
    except Exception as e:
        # rollback current transaction if there is an error
        cnxn.rollback()
        raise e
    print(msg)

# insert data into olist_customers table
write_df(cust_df, "olist_customers", "olist_customers successfully updated.")


olist_customers successfully updated.


In [ ]:
cursor.close()
cnxn.close()

The `olist_geolocation_dataset` contains latitude and longitude points, city and state for the zipcodes. Every customer in the customer dataset has a zipcode associated as their location within the state and city.

In [ ]:
# import dataset
geo_df = pd.read_csv('data/olist_geolocation_dataset.csv')

# view dataset
geo_df.head()

### olist_order_reviews_dataset.csv

In [ ]:
# review_id: unique review identifier
# order_id: unique order identifier
# review_score: Note ranging from 1 to 5 given by the customer on a satisfaction survey
# review_comment_title: Comment title from the review left by the customer, in Portuguese
# review_comment_message: Comment message from the review left by the customer, in Portuguese
# review_creation_date: Shows the date in which the satisfaction survey was sent to the customer
# review_answer_timestamp: Shows satisfaction survey answer timestamp

# Read in olist_order_reviews_dataset.csv as pandas dataframe and parse dates of 
# "review_creation_date" and "review_answer_timestamp" columns.
parse_dates = ["review_creation_date", "review_answer_timestamp"]
order_reviews_data = pd.read_csv("olist_order_reviews_dataset.csv",\
                                 infer_datetime_format = True, parse_dates = parse_dates)

# Drop duplicates from review_id column.
order_reviews_data.drop_duplicates(subset = ['review_id'], inplace = True)

# Preview the first 5 lines of the loaded data 
order_reviews_data.head()

In [ ]:
# Count number of entries in each column
order_reviews_data.count()

In [ ]:
# Count number of unique entries
order_reviews_data.nunique()

In [ ]:
# Check for null values
order_reviews_data.isnull().sum()

In [ ]:
# Drop columns with null values
order_reviews_data.drop(columns = ["review_comment_title", "review_comment_message"], inplace = True)

In [ ]:
# Check datatypes
order_reviews_data.dtypes

In [ ]:
# Change column to datetime
order_reviews_data['review_answer_timestamp'] = order_reviews_data['review_answer_timestamp'].dt.date
order_reviews_data.head()

In [ ]:
# Check datatypes
order_reviews_data.dtypes

olist_order_payments_dataset.csv

In [ ]:
# order_id: unique order identifier
# payment_sequential: a customer may pay an order with more than one payment method. If he does so, a sequence will be created to
# payment_type: method of payment chosen by the customer
# payment_installments: number of installments chosen by the customer
# payment_value: transaction value

# Boleto payments: Boleto is an official (regulated by the Central Bank of Brazil) payment method in Brazil. 
# To complete a transaction, customers receive a voucher stating the amount to pay for services or goods. 
# Customers then pay the boleto before its expiration date in one of several different methods, including at authorized agencies or banks, ATMs, or online bank portals. 
# You will receive payment confirmation after 1 business day, while funds will be available for payout 2 business days after payment confirmation.
# https://stripe.com/docs/payments/boleto 
# https://www.rapyd.net/blog/what-is-boleto-everything-you-need-to-know/ 

# Read in olist_order_payments_dataset.csv and make pandas dataframe
order_payment_data = pd.read_csv("C:/Users/Mavis Luo/Downloads/Final Year Project/Dataset/olist_order_payments_dataset.csv")

# Drop duplicates from order_id column
order_payment_data.drop_duplicates(subset = ['order_id'], inplace = True)

# Set index as order_id
order_payment_data.set_index("order_id", inplace = True)

# Print dataframe
order_payment_data.head()

In [ ]:
# Count rows of each column
order_payment_data.count()

In [ ]:
# Count number of unique entries
order_payment_data.nunique()

In [ ]:
# Print dataframe
order_payment_data.head()

In [ ]:
# Check for null values
order_payment_data.isnull().sum()

In [ ]:
# Check datatypes
order_payment_data.dtypes

In [ ]:
df_comments = olist_order_reviews.loc[:, ['review_score', 'review_comment_message']]
df_comments = df_comments.dropna(subset=['review_comment_message'])
df_comments = df_comments.reset_index(drop=True)
print(f'Dataset shape: {df_comments.shape}')
df_comments.columns = ['score', 'comment']
df_comments.head()